# Data Cleaning

Preparing SaverLife Saver Transaction History Data for [AWS RDS](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Welcome.html)

*Note*: Assumes [`xlrd`](https://xlrd.readthedocs.io/en/latest/) is installed

In [ ]:
import pandas as pd

class Transactions:
    """
    SaverLife Savers Transaction Data
    """

    def __init__(self, parent_grandparent_categories, transaction_data_tables):
        """
        Initialize Transactions class with data inputs

        Args:
            parent_grandparent_categories: Plaid API transaction categories
            transaction_data_tables: Plaid transaction data (stored as XLSX file)        
        """

        self.parent_grandparent_categories = parent_grandparent_categories
        self.transaction_data_tables = transaction_data_tables
        
    def clean_and_save(self, output_file):
        """ 
        Method to merge/clean transaction data tables and save to file
        
        Args:
            output_file: Local file path to save cleaned transction data

        Returns:
            Success statement upon completion of save 
        """

        self.output_file = output_file

        # Read category data using pandas `read_csv` function
        self.categories = pd.read_csv(self.parent_grandparent_categories)

        # Initialize list with features of interest to select from categories data
        self.categories_of_interest = ["plaid_category_id",
                                       "category_name",
                                       "parent_category_name",
                                       "grandparent_category_name"]
        
        # Read PFA sheet of transaction_data_tables Excel notebook, dropping duplicates
        self.PFA = pd.read_excel(self.transaction_data_tables, 
                                 "PFA").drop_duplicates()
        
        # Read BA sheet of transaction_data_tables Excel notebook, dropping duplicates
        self.BA = pd.read_excel(self.transaction_data_tables, 
                                "BA").drop_duplicates()
        
        # Read PMT sheet of transaction_data_tables Excel notebook
        self.PMT = pd.read_excel(self.transaction_data_tables, 
                                 "PMT")

        # Merge PMT table with Plaid spending categories
        self.PMT_categories = pd.merge(self.PMT, 
                                       self.categories[self.categories_of_interest], 
                                       left_on="category_id", 
                                       right_on="plaid_category_id")
        
        # Merge PMT_categories with BA table
        self.PMT_categories_BA = pd.merge(self.PMT_categories, 
                                          self.BA, 
                                          left_on="bank_account_id", 
                                          right_on="id")
        
        # Merge PMT_categories_BA with PFA table
        self.PMT_categories_BA_PFA = pd.merge(self.PMT_categories_BA, 
                                              self.PFA, 
                                              left_on="plaid_financial_authentication_id", 
                                              right_on="id")
        
        # Save PMT_categories_BA_PFA to local CSV file at output_file
        self.PMT_categories_BA_PFA.to_csv(self.output_file)

        # Return success statement to inform user of completion
        print("It worked")

In [ ]:
SaverLifeSavers(parent_grandparent_categories="/home/alex/Downloads/parent_grandparent_categories copy.csv",
                transaction_data_tables="/home/alex/Downloads/PFA_BA_PMT_Table_examples.xlsx").clean_and_save("transaction_data.csv")